In [50]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import glob
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_files
import scipy as sp
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
import logging

In [48]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [38]:
#import pickle file of langauge data
lang_semantics = pickle.load( open( "language_data/data.p", "rb" ) )

In [39]:
#Make lists the same size
for key in lang_semantics:
    while len(lang_semantics[key]) > 80:
        lang_semantics[key].pop()
    while len(lang_semantics[key]) < 80:
        lang_semantics[key].append('NaN')
    print(key, " len ", len(lang_semantics[key]))


C#  len  80
C++  len  80
Haskell  len  80
C  len  80
Scheme  len  80
Scala  len  80
Ruby  len  80
Python  len  80
HicEst  len  80
JavaScript  len  80
Java  len  80
Perl  len  80
PHP  len  80
OCaml  len  80
Clojure  len  80


In [40]:
df = pd.DataFrame(lang_semantics)
#renaming C, C#, and C++ because count_vectorizer doesn't recognize them as features
df=df.rename(columns = {'C#':'C_Sharp', 'C++':'CPP', 'C':'C_Lang'})
df.head()

,C_Lang,C_Sharp,CPP,Clojure,Haskell,HicEst,Java,JavaScript,OCaml,PHP,Perl,Python,Ruby,Scala,Scheme
0,// Standard input-output streams#include <stdi...,using System;using System.Linq; class Program{...,// Standard input-output streams#include <iost...,(println (+ (Integer/parseInt (read-line)) (In...,main = print . sum . map read . words =<< getLine,"DLG(Edit=A, DNum, MIn=-1000, MAx=1000, E=B, DN...",import java.util.*; public class Sum2 { public...,"process.openStdin().on ( 'data', function (lin...","Scanf.scanf ""%d %d"" (fun a b -> Printf.printf ...","fscanf(STDIN, ""%d %d\n"", $a, $b); //Reads 2 nu...","my ($a,$b) = split(' ', scalar(<STDIN>));print...",try: raw_inputexcept: raw_input = input print(...,puts gets.not_nil!.split.map(&.to_i).sum,"println(readLine().split("" "").map(_.toInt).sum)","(+ (read-number 1 ""value for A"") (read-number..."
1,/* double linked list */#include <stdio.h>#inc...,public static ulong Fib(uint n) { return (n <...,#include <iostream>#include <list> int main ()...,(ns double-list) (defprotocol PDoubleList (get...,import qualified Data.Map as M type NodeID = M...,REAL :: Fibonacci(10) Fibonacci = ($==2) + Fib...,import java.math.BigInteger; public class Benf...,"function fib(n) { return function(n,a,b) { ret...",open Num let fib = let rec fib_aux f0 f1 = fu...,function fibIter($n) { if ($n < 2) { return $n...,#!/usr/bin/perluse strict ;use warnings ;use P...,from __future__ import divisionfrom itertools ...,EXPECTED = (1..9).map{|d| Math.log10(1+1.0/d)}...,package benford import java.math.BigInteger in...,(define (fib n) (if (< n 2) n (+ (fib (- n 2)...
2,#include <stdio.h>#include <stdlib.h>#include ...,using System;using System.Collections.Generic;...,"//to cope with the big numbers , I used the Cl...",(defn fibs [] (map first (iterate (fn [[a b]] ...,import qualified Data.Map as Mimport Data.Char...,"OPEN(FIle='test.txt') 1 DLG(NameEdit=x0, DNum=...",import java.awt.*;import javax.swing.*; public...,function Person(name) { var candidateIndex = ...,"fun fib (0, x1, x2) = x2 | (n, x1, x2) = fib (...",<html><head><title>n x n Queen solving progra...,use ntheory qw/is_prime/;use Imager; my $n = s...,# coding=UTF-8from __future__ import print_fun...,"require 'prime' def cell(n, x, y, start=1) y, ...",package ulam object Ulam { fun generate(n: Int...,";; 1)(define \a (first (string->unicode ""a""))..."
3,#include <stdio.h>#include <stdint.h>#include...,using System;using System.Collections.Generic;...,#include <cmath>#include <iostream>#include <s...,(defn findRoots [f start stop step eps] (fil...,import Data.Listimport Data.Numbers.Primes -- ...,"WINDOW(WINdowhandle=wh, BaCkcolor=0, TItle=""Ro...",public static long itFibN(int n){ if (n < 2) r...,"// This function notation is sorta new, but u...","let men = [ ""abe"", [""abi"";""eve"";""cath"";""ivy"";""...","<?php$lower = range('a', 'z');var_dump($lower);?>",sub fib_iter { my $n = shift; use bigint try =...,from math import * def analytic_fibonacci(n): ...,"def fib(n, sequence=[1]) n.times do current_nu...",package fibonacci enum class Fibonacci { ITERA...,"(string-size ""Hello world"")"
4,"// Fibonacci sequence, recursive versionfun f...",using System;using System.Collections.Generic...,#include <iostream> int main(){ unsigned int a...,(defn ints-from [n] (cons n (lazy-seq (ints-fr...,[floor(0.01+(1/p**n+p**n)/sqrt 5)|let p=(1+sqr...,"LEN(""1 character == 1 byte"") ! 21",public class Roots { public interface Function...,"function queenPuzzle(rows, columns) { if (row...",let bracket u v = ((u > 0.0) && (v < 0.0)) || ...,"<?phpforeach (array('møøse', 'Unicode', 'J̲o̲s...",#!/usr/bin/env perl use strict;use warnings;us...,"import copy guyprefers = { 'abe': ['abi', 'eve...",class Person def initialize(name) @name = name...,import java.util.* class People(val map: Map<S...,(define (next-row row) (map + (cons 0 row) (ap...


In [41]:
vectorizer = CountVectorizer(min_df=1)

In [10]:
X_train = vectorizer.fit_transform(df)
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d" %(num_samples, num_features))

#samples: 15, #features: 15


In [11]:
vectorizer.get_feature_names()

['c_lang',
 'c_sharp',
 'clojure',
 'cpp',
 'haskell',
 'hicest',
 'java',
 'javascript',
 'ocaml',
 'perl',
 'php',
 'python',
 'ruby',
 'scala',
 'scheme']

In [37]:
# loading test data
test_data = load_files('test_data/', load_content=True,
                      encoding='UTF-8')
test_data.filenames

array(['test_data/test/2', 'test_data/test/3', 'test_data/test/19',
       'test_data/test/11', 'test_data/test/24', 'test_data/test/22',
       'test_data/test/6', 'test_data/test/4', 'test_data/test/28',
       'test_data/test/21', 'test_data/test/31', 'test_data/test/14',
       'test_data/test/25', 'test_data/test/17', 'test_data/test/8',
       'test_data/test/32', 'test_data/test/30', 'test_data/test/10',
       'test_data/test/9', 'test_data/test/15', 'test_data/test/13',
       'test_data/test/26', 'test_data/test/7', 'test_data/test/27',
       'test_data/test/18', 'test_data/test/16', 'test_data/test/5',
       'test_data/test/12', 'test_data/test/1', 'test_data/test/29',
       'test_data/test/23', 'test_data/test/20'], 
      dtype='<U17')

In [42]:
df.describe()

,C_Lang,C_Sharp,CPP,Clojure,Haskell,HicEst,Java,JavaScript,OCaml,PHP,Perl,Python,Ruby,Scala,Scheme
count,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80
unique,80,80,80,80,80,80,80,80,80,80,80,80,80,80,80
top,"// Fibonacci sequence, recursive versionfun f...",using System;using System.Collections.Generic;...,#include <string>#include <iostream> using nam...,(defn mkdirp [path] (let [dir (java.io.File. p...,import Data.Listimport Data.Timeimport Data.Ti...,"REAL :: a(1000) a = 1 / $^2 WRITE(ClipBoard, F...",public class Roots { public interface Function...,function raze(a) { // like .join('') except pr...,class animal = object (self) (*functions go he...,<?php // fputcsv() requires at least PHP 5.1....,sub multiply { return $_[0] * $_[1] },class Animal: pass #functions go here... class...,# Calculate Pi using the Arithmetic Geometric ...,import java.util.regex.Pattern.quotedef strip1...,"; auxiliary functions : range, filter(define (..."
freq,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [43]:
df.['c_length'] = df.[]

,C_Lang,C_Sharp,CPP,Clojure,Haskell,HicEst,Java,JavaScript,OCaml,PHP,Perl,Python,Ruby,Scala,Scheme
0,// Standard input-output streams#include <stdi...,using System;using System.Linq; class Program{...,// Standard input-output streams#include <iost...,(println (+ (Integer/parseInt (read-line)) (In...,main = print . sum . map read . words =<< getLine,"DLG(Edit=A, DNum, MIn=-1000, MAx=1000, E=B, DN...",import java.util.*; public class Sum2 { public...,"process.openStdin().on ( 'data', function (lin...","Scanf.scanf ""%d %d"" (fun a b -> Printf.printf ...","fscanf(STDIN, ""%d %d\n"", $a, $b); //Reads 2 nu...","my ($a,$b) = split(' ', scalar(<STDIN>));print...",try: raw_inputexcept: raw_input = input print(...,puts gets.not_nil!.split.map(&.to_i).sum,"println(readLine().split("" "").map(_.toInt).sum)","(+ (read-number 1 ""value for A"") (read-number..."
1,/* double linked list */#include <stdio.h>#inc...,public static ulong Fib(uint n) { return (n <...,#include <iostream>#include <list> int main ()...,(ns double-list) (defprotocol PDoubleList (get...,import qualified Data.Map as M type NodeID = M...,REAL :: Fibonacci(10) Fibonacci = ($==2) + Fib...,import java.math.BigInteger; public class Benf...,"function fib(n) { return function(n,a,b) { ret...",open Num let fib = let rec fib_aux f0 f1 = fu...,function fibIter($n) { if ($n < 2) { return $n...,#!/usr/bin/perluse strict ;use warnings ;use P...,from __future__ import divisionfrom itertools ...,EXPECTED = (1..9).map{|d| Math.log10(1+1.0/d)}...,package benford import java.math.BigInteger in...,(define (fib n) (if (< n 2) n (+ (fib (- n 2)...
2,#include <stdio.h>#include <stdlib.h>#include ...,using System;using System.Collections.Generic;...,"//to cope with the big numbers , I used the Cl...",(defn fibs [] (map first (iterate (fn [[a b]] ...,import qualified Data.Map as Mimport Data.Char...,"OPEN(FIle='test.txt') 1 DLG(NameEdit=x0, DNum=...",import java.awt.*;import javax.swing.*; public...,function Person(name) { var candidateIndex = ...,"fun fib (0, x1, x2) = x2 | (n, x1, x2) = fib (...",<html><head><title>n x n Queen solving progra...,use ntheory qw/is_prime/;use Imager; my $n = s...,# coding=UTF-8from __future__ import print_fun...,"require 'prime' def cell(n, x, y, start=1) y, ...",package ulam object Ulam { fun generate(n: Int...,";; 1)(define \a (first (string->unicode ""a""))..."
3,#include <stdio.h>#include <stdint.h>#include...,using System;using System.Collections.Generic;...,#include <cmath>#include <iostream>#include <s...,(defn findRoots [f start stop step eps] (fil...,import Data.Listimport Data.Numbers.Primes -- ...,"WINDOW(WINdowhandle=wh, BaCkcolor=0, TItle=""Ro...",public static long itFibN(int n){ if (n < 2) r...,"// This function notation is sorta new, but u...","let men = [ ""abe"", [""abi"";""eve"";""cath"";""ivy"";""...","<?php$lower = range('a', 'z');var_dump($lower);?>",sub fib_iter { my $n = shift; use bigint try =...,from math import * def analytic_fibonacci(n): ...,"def fib(n, sequence=[1]) n.times do current_nu...",package fibonacci enum class Fibonacci { ITERA...,"(string-size ""Hello world"")"
4,"// Fibonacci sequence, recursive versionfun f...",using System;using System.Collections.Generic...,#include <iostream> int main(){ unsigned int a...,(defn ints-from [n] (cons n (lazy-seq (ints-fr...,[floor(0.01+(1/p**n+p**n)/sqrt 5)|let p=(1+sqr...,"LEN(""1 character == 1 byte"") ! 21",public class Roots { public interface Function...,"function queenPuzzle(rows, columns) { if (row...",let bracket u v = ((u > 0.0) && (v < 0.0)) || ...,"<?phpforeach (array('møøse', 'Unicode', 'J̲o̲s...",#!/usr/bin/env perl use strict;use warnings;us...,"import copy guyprefers = { 'abe': ['abi', 'eve...",class Person def initialize(name) @name = name...,import java.util.* class People(val map: Map<S...,(define (next-row row) (map + (cons 0 row) (ap...


In [51]:
background_corpus = TextCorpus(input=lang_semantics)

AttributeError: 'dict' object has no attribute 'seek'